# 🚀 Advanced Usage & Production Deployment

## Overview

This notebook covers advanced topics for deploying MGPT-Eval in production environments, optimizing performance, and integrating with existing systems.

## 🏭 Production Deployment

### Production-Ready Configuration

```yaml
# production_config.yaml
job:
  name: "production_mgpt_eval_${TIMESTAMP}"  # Dynamic naming
  output_dir: "/data/mgpt_eval_results"
  random_seed: 42

model_api:
  base_url: "${MGPT_API_URL}"               # Environment variable
  batch_size: 64                            # Optimized for production
  timeout: 300
  max_retries: 5                            # More resilient

# Memory-optimized settings
data_processing:
  output_format: "csv"                      # Efficient format
  max_sequence_length: 512

embedding_generation:
  batch_size: 32
  save_interval: 50                         # Frequent checkpoints
  resume_from_checkpoint: true

# Production logging
logging:
  level: "INFO"
  console_level: "WARNING"                  # Minimal console output
  file: "/logs/mgpt_eval_${TIMESTAMP}.log"
```

### Docker Deployment

#### Dockerfile
```dockerfile
FROM python:3.9-slim

# Install system dependencies
RUN apt-get update && apt-get install -y \
    curl \
    && rm -rf /var/lib/apt/lists/*

# Set working directory
WORKDIR /app

# Copy requirements and install
COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

# Copy application code
COPY . .

# Create output directories
RUN mkdir -p /data/outputs /data/logs

# Set environment variables
ENV PYTHONPATH=/app
ENV MGPT_EVAL_OUTPUT_DIR=/data/outputs
ENV MGPT_EVAL_LOG_DIR=/data/logs

# Health check
HEALTHCHECK --interval=30s --timeout=10s --start-period=5s --retries=3 \
  CMD python -c "import sys; sys.exit(0)"

# Default command
CMD ["python", "main.py", "--help"]
```

#### Docker Compose
```yaml
# docker-compose.yml
version: '3.8'

services:
  mgpt-eval:
    build: .
    volumes:
      - ./data:/data
      - ./configs:/app/configs
      - ./outputs:/data/outputs
    environment:
      - MGPT_API_URL=http://mgpt-server:8000
      - TIMESTAMP=${TIMESTAMP:-$(date +%Y%m%d_%H%M%S)}
    depends_on:
      - mgpt-server
    command: >
      python main.py run-all 
      --config /app/configs/production_config.yaml

  mgpt-server:
    image: your-mgpt-server:latest
    ports:
      - "8000:8000"
    environment:
      - MODEL_PATH=/models/mgpt
    volumes:
      - ./models:/models
```

#### Deployment Commands
```bash
# Build and deploy
export TIMESTAMP=$(date +%Y%m%d_%H%M%S)
docker-compose up --build

# Run specific job
docker-compose run mgpt-eval python main.py run-all \
  --config /app/configs/diabetes_evaluation.yaml

# Check logs
docker-compose logs -f mgpt-eval
```

### Kubernetes Deployment

#### Job Template
```yaml
# k8s-job.yaml
apiVersion: batch/v1
kind: Job
metadata:
  name: mgpt-eval-job
  labels:
    app: mgpt-eval
spec:
  template:
    metadata:
      labels:
        app: mgpt-eval
    spec:
      restartPolicy: Never
      containers:
      - name: mgpt-eval
        image: your-registry/mgpt-eval:latest
        resources:
          requests:
            memory: "4Gi"
            cpu: "2"
          limits:
            memory: "8Gi"
            cpu: "4"
        env:
        - name: MGPT_API_URL
          value: "http://mgpt-service:8000"
        - name: JOB_NAME
          value: "evaluation-$(date +%Y%m%d-%H%M%S)"
        volumeMounts:
        - name: data-volume
          mountPath: /data
        - name: config-volume
          mountPath: /app/configs
        command:
        - python
        - main.py
        - run-all
        - --config
        - /app/configs/production_config.yaml
      volumes:
      - name: data-volume
        persistentVolumeClaim:
          claimName: mgpt-eval-data
      - name: config-volume
        configMap:
          name: mgpt-eval-config
```

#### ConfigMap
```yaml
# k8s-configmap.yaml
apiVersion: v1
kind: ConfigMap
metadata:
  name: mgpt-eval-config
data:
  production_config.yaml: |
    job:
      name: "production_eval"
      output_dir: "/data/outputs"
    model_api:
      base_url: "http://mgpt-service:8000"
      batch_size: 64
    # ... rest of configuration
```

## ⚡ Performance Optimization

### Large-Scale Data Processing

#### Chunked Processing for Very Large Datasets
```python
# scripts/chunk_processor.py
import pandas as pd
import os
import subprocess
from pathlib import Path

def process_large_dataset(dataset_path, chunk_size=5000, output_dir="outputs"):
    """Process large dataset in chunks to avoid memory issues."""
    
    # Load and split dataset
    df = pd.read_csv(dataset_path)
    total_chunks = len(df) // chunk_size + (1 if len(df) % chunk_size else 0)
    
    chunk_results = []
    
    for i in range(total_chunks):
        start_idx = i * chunk_size
        end_idx = min((i + 1) * chunk_size, len(df))
        
        # Create chunk
        chunk = df[start_idx:end_idx]
        chunk_file = f"chunk_{i}.csv"
        chunk.to_csv(chunk_file, index=False)
        
        # Process chunk
        config = create_chunk_config(chunk_file, f"chunk_{i}_eval")
        result = subprocess.run([
            "python", "main.py", "run-all", "--config", config
        ], capture_output=True, text=True)
        
        if result.returncode == 0:
            chunk_results.append(f"outputs/chunk_{i}_eval")
            print(f"Chunk {i+1}/{total_chunks} completed successfully")
        else:
            print(f"Chunk {i+1}/{total_chunks} failed: {result.stderr}")
        
        # Cleanup
        os.remove(chunk_file)
    
    # Aggregate results
    aggregate_chunk_results(chunk_results, output_dir)

def create_chunk_config(chunk_file, job_name):
    """Create configuration for chunk processing."""
    config = f"""
input:
  dataset_path: "{chunk_file}"
  split_ratio: 0.8

job:
  name: "{job_name}"
  output_dir: "outputs"

# Optimized for chunks
model_api:
  batch_size: 32
  timeout: 300

pipeline_stages:
  embeddings: true
  classification: true
  evaluation: true
  target_word_eval: false  # Skip for chunks
  summary_report: true
  method_comparison: false

data_processing:
  output_format: "csv"  # Memory efficient
"""
    config_file = f"chunk_config_{job_name}.yaml"
    with open(config_file, 'w') as f:
        f.write(config)
    return config_file
```

#### Parallel Processing
```python
# scripts/parallel_processor.py
import concurrent.futures
import subprocess
from pathlib import Path

def run_parallel_evaluation(configs, max_workers=4):
    """Run multiple evaluations in parallel."""
    
    def run_single_eval(config_path):
        """Run single evaluation job."""
        result = subprocess.run([
            "python", "main.py", "run-all", "--config", config_path
        ], capture_output=True, text=True)
        
        return {
            "config": config_path,
            "success": result.returncode == 0,
            "output": result.stdout,
            "error": result.stderr
        }
    
    # Run evaluations in parallel
    with concurrent.futures.ProcessPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(run_single_eval, config): config for config in configs}
        
        results = []
        for future in concurrent.futures.as_completed(futures):
            config = futures[future]
            try:
                result = future.result()
                results.append(result)
                print(f"Completed: {config} - {'Success' if result['success'] else 'Failed'}")
            except Exception as exc:
                print(f"Failed: {config} generated an exception: {exc}")
    
    return results

# Usage example
configs = [
    "configs/diabetes_eval.yaml",
    "configs/cardiovascular_eval.yaml", 
    "configs/respiratory_eval.yaml"
]

results = run_parallel_evaluation(configs, max_workers=3)
```

### Resource Monitoring & Optimization

#### Memory Monitoring
```python
# scripts/resource_monitor.py
import psutil
import time
import json
from datetime import datetime

class ResourceMonitor:
    def __init__(self, log_file="resource_usage.log"):
        self.log_file = log_file
        self.monitoring = False
    
    def start_monitoring(self, interval=30):
        """Start monitoring system resources."""
        self.monitoring = True
        
        with open(self.log_file, 'w') as f:
            f.write("timestamp,cpu_percent,memory_percent,memory_used_gb,disk_usage_percent\n")
        
        while self.monitoring:
            stats = {
                "timestamp": datetime.now().isoformat(),
                "cpu_percent": psutil.cpu_percent(interval=1),
                "memory_percent": psutil.virtual_memory().percent,
                "memory_used_gb": psutil.virtual_memory().used / (1024**3),
                "disk_usage_percent": psutil.disk_usage('/').percent
            }
            
            with open(self.log_file, 'a') as f:
                f.write(f"{stats['timestamp']},{stats['cpu_percent']},{stats['memory_percent']},{stats['memory_used_gb']:.2f},{stats['disk_usage_percent']}\n")
            
            time.sleep(interval)
    
    def stop_monitoring(self):
        """Stop monitoring."""
        self.monitoring = False

# Usage in production
monitor = ResourceMonitor()
monitor.start_monitoring(interval=60)  # Monitor every minute
```

#### Adaptive Batch Sizing
```python
# utils/adaptive_batching.py
import psutil
import time

class AdaptiveBatchSizer:
    def __init__(self, initial_batch_size=32, memory_threshold=80):
        self.current_batch_size = initial_batch_size
        self.memory_threshold = memory_threshold
        self.performance_history = []
    
    def get_optimal_batch_size(self):
        """Get optimal batch size based on current system state."""
        memory_usage = psutil.virtual_memory().percent
        
        if memory_usage > self.memory_threshold:
            # Reduce batch size if memory usage is high
            self.current_batch_size = max(1, self.current_batch_size // 2)
            print(f"Memory usage {memory_usage}% > {self.memory_threshold}%, reducing batch size to {self.current_batch_size}")
        elif memory_usage < self.memory_threshold - 20:
            # Increase batch size if memory usage is low
            self.current_batch_size = min(128, self.current_batch_size * 2)
            print(f"Memory usage {memory_usage}% is low, increasing batch size to {self.current_batch_size}")
        
        return self.current_batch_size
    
    def record_performance(self, batch_size, processing_time, memory_peak):
        """Record performance metrics for optimization."""
        self.performance_history.append({
            'batch_size': batch_size,
            'processing_time': processing_time,
            'memory_peak': memory_peak,
            'throughput': batch_size / processing_time
        })
        
        # Keep only recent history
        if len(self.performance_history) > 100:
            self.performance_history = self.performance_history[-100:]
```

## 🔄 Automated Workflows

### Scheduled Evaluations

#### Cron-based Scheduling
```bash
# /etc/cron.d/mgpt-eval
# Run daily evaluation at 2 AM
0 2 * * * mgpt-user cd /opt/mgpt-eval && python scripts/daily_evaluation.py

# Run weekly comprehensive evaluation on Sundays at midnight
0 0 * * 0 mgpt-user cd /opt/mgpt-eval && python scripts/weekly_evaluation.py
```

#### Daily Evaluation Script
```python
# scripts/daily_evaluation.py
import os
import subprocess
import smtplib
from email.mime.text import MimeText
from email.mime.multipart import MimeMultipart
from datetime import datetime
import json

def run_daily_evaluation():
    """Run daily model evaluation and send report."""
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    job_name = f"daily_eval_{timestamp}"
    
    # Configuration for daily evaluation
    config = {
        "job": {
            "name": job_name,
            "output_dir": "/data/daily_evaluations"
        },
        "input": {
            "dataset_path": "/data/latest/medical_claims.csv"
        },
        "pipeline_stages": {
            "embeddings": True,
            "classification": True,
            "evaluation": True,
            "target_word_eval": True,
            "summary_report": True,
            "method_comparison": True
        }
    }
    
    # Save configuration
    config_file = f"daily_config_{timestamp}.yaml"
    with open(config_file, 'w') as f:
        yaml.dump(config, f)
    
    try:
        # Run evaluation
        result = subprocess.run([
            "python", "main.py", "run-all", "--config", config_file
        ], capture_output=True, text=True, timeout=3600)  # 1 hour timeout
        
        if result.returncode == 0:
            # Load results
            summary_file = f"/data/daily_evaluations/{job_name}/summary/pipeline_summary.json"
            with open(summary_file) as f:
                results = json.load(f)
            
            send_success_report(job_name, results)
        else:
            send_failure_report(job_name, result.stderr)
    
    except subprocess.TimeoutExpired:
        send_failure_report(job_name, "Evaluation timed out after 1 hour")
    
    except Exception as e:
        send_failure_report(job_name, str(e))
    
    finally:
        # Cleanup
        os.remove(config_file)

def send_success_report(job_name, results):
    """Send success notification with results."""
    best_accuracy = results.get('best_results', {}).get('embedding_method', {}).get('accuracy', 0)
    
    subject = f"✅ Daily MGPT Evaluation Success - {job_name}"
    body = f"""
Daily MGPT evaluation completed successfully!

Job: {job_name}
Best Accuracy: {best_accuracy:.1%}
Total Samples: {results.get('data_summary', {}).get('total_samples', 'N/A')}

Full results available at: /data/daily_evaluations/{job_name}/
"""
    
    send_email(subject, body)

def send_failure_report(job_name, error):
    """Send failure notification."""
    subject = f"❌ Daily MGPT Evaluation Failed - {job_name}"
    body = f"""
Daily MGPT evaluation failed!

Job: {job_name}
Error: {error}

Please check the logs for more details.
"""
    
    send_email(subject, body)

def send_email(subject, body):
    """Send email notification."""
    # Email configuration from environment variables
    smtp_server = os.getenv('SMTP_SERVER', 'localhost')
    smtp_port = int(os.getenv('SMTP_PORT', '587'))
    smtp_user = os.getenv('SMTP_USER')
    smtp_password = os.getenv('SMTP_PASSWORD')
    to_emails = os.getenv('NOTIFICATION_EMAILS', '').split(',')
    
    if not to_emails or not smtp_user:
        print("Email configuration not found, skipping notification")
        return
    
    msg = MimeMultipart()
    msg['From'] = smtp_user
    msg['To'] = ', '.join(to_emails)
    msg['Subject'] = subject
    
    msg.attach(MimeText(body, 'plain'))
    
    try:
        server = smtplib.SMTP(smtp_server, smtp_port)
        server.starttls()
        server.login(smtp_user, smtp_password)
        text = msg.as_string()
        server.sendmail(smtp_user, to_emails, text)
        server.quit()
        print("Notification email sent successfully")
    except Exception as e:
        print(f"Failed to send email: {e}")

if __name__ == "__main__":
    run_daily_evaluation()
```

### CI/CD Integration

#### GitHub Actions Workflow
```yaml
# .github/workflows/model-evaluation.yml
name: Model Evaluation Pipeline

on:
  schedule:
    - cron: '0 6 * * *'  # Daily at 6 AM UTC
  workflow_dispatch:     # Manual trigger
    inputs:
      dataset_path:
        description: 'Path to dataset file'
        required: true
        default: 'data/medical_claims.csv'
      target_codes:
        description: 'Target codes (comma-separated)'
        required: true
        default: 'E119,I10,N6320'

jobs:
  evaluate-model:
    runs-on: ubuntu-latest
    
    strategy:
      matrix:
        evaluation_type: ["embeddings", "target_words", "full_pipeline"]
    
    steps:
    - name: Checkout code
      uses: actions/checkout@v3
    
    - name: Set up Python
      uses: actions/setup-python@v4
      with:
        python-version: '3.9'
    
    - name: Install dependencies
      run: |
        pip install -r requirements.txt
    
    - name: Download test data
      run: |
        # Download or generate test data
        wget ${{ secrets.TEST_DATA_URL }} -O data/test_claims.csv
    
    - name: Start mock MGPT server
      run: |
        python tests/fake_api_server.py &
        sleep 10  # Wait for server to start
    
    - name: Run evaluation
      env:
        MGPT_API_URL: http://localhost:8000
      run: |
        python main.py run-all --config configs/templates/04_full_pipeline.yaml
    
    - name: Upload results
      uses: actions/upload-artifact@v3
      with:
        name: evaluation-results-${{ matrix.evaluation_type }}
        path: outputs/
    
    - name: Publish results
      if: success()
      run: |
        # Parse results and create summary
        python scripts/create_github_summary.py
    
    - name: Notify on failure
      if: failure()
      uses: 8398a7/action-slack@v3
      with:
        status: failure
        text: 'MGPT Evaluation failed for ${{ matrix.evaluation_type }}'
      env:
        SLACK_WEBHOOK_URL: ${{ secrets.SLACK_WEBHOOK_URL }}
```

#### Jenkins Pipeline
```groovy
// Jenkinsfile
pipeline {
    agent any
    
    parameters {
        choice(
            name: 'EVALUATION_TYPE',
            choices: ['full_pipeline', 'embeddings_only', 'target_words_only'],
            description: 'Type of evaluation to run'
        )
        string(
            name: 'DATASET_PATH',
            defaultValue: 'data/medical_claims.csv',
            description: 'Path to dataset file'
        )
    }
    
    environment {
        MGPT_API_URL = credentials('mgpt-api-url')
        NOTIFICATION_EMAILS = credentials('notification-emails')
    }
    
    stages {
        stage('Checkout') {
            steps {
                checkout scm
            }
        }
        
        stage('Setup') {
            steps {
                sh '''
                    python -m venv venv
                    source venv/bin/activate
                    pip install -r requirements.txt
                '''
            }
        }
        
        stage('Data Validation') {
            steps {
                script {
                    def dataExists = fileExists(params.DATASET_PATH)
                    if (!dataExists) {
                        error "Dataset file not found: ${params.DATASET_PATH}"
                    }
                }
            }
        }
        
        stage('Run Evaluation') {
            steps {
                sh '''
                    source venv/bin/activate
                    python main.py run-all --config configs/templates/${EVALUATION_TYPE}.yaml
                '''
            }
        }
        
        stage('Archive Results') {
            steps {
                archiveArtifacts artifacts: 'outputs/**/*', fingerprint: true
                publishHTML([
                    allowMissing: false,
                    alwaysLinkToLastBuild: true,
                    keepAll: true,
                    reportDir: 'outputs',
                    reportFiles: '*/summary/pipeline_summary.json',
                    reportName: 'Evaluation Results'
                ])
            }
        }
    }
    
    post {
        success {
            emailext (
                subject: "✅ MGPT Evaluation Success - ${env.BUILD_NUMBER}",
                body: """MGPT evaluation completed successfully!
                
                Build: ${env.BUILD_NUMBER}
                Type: ${params.EVALUATION_TYPE}
                Dataset: ${params.DATASET_PATH}
                
                View results: ${env.BUILD_URL}""",
                to: "${env.NOTIFICATION_EMAILS}"
            )
        }
        
        failure {
            emailext (
                subject: "❌ MGPT Evaluation Failed - ${env.BUILD_NUMBER}",
                body: """MGPT evaluation failed!
                
                Build: ${env.BUILD_NUMBER}
                Type: ${params.EVALUATION_TYPE}
                
                View logs: ${env.BUILD_URL}""",
                to: "${env.NOTIFICATION_EMAILS}"
            )
        }
        
        always {
            cleanWs()
        }
    }
}
```

## 📊 Integration with Analytics Platforms

### MLflow Integration

```python
# utils/mlflow_integration.py
import mlflow
import mlflow.sklearn
import json
from pathlib import Path

class MLflowLogger:
    def __init__(self, experiment_name="mgpt-eval"):
        mlflow.set_experiment(experiment_name)
    
    def log_evaluation_run(self, job_name, config, results_dir):
        """Log complete evaluation run to MLflow."""
        
        with mlflow.start_run(run_name=job_name):
            # Log configuration parameters
            self._log_config_params(config)
            
            # Log metrics from all methods
            self._log_classification_metrics(results_dir)
            self._log_target_word_metrics(results_dir)
            
            # Log models
            self._log_trained_models(results_dir)
            
            # Log artifacts
            mlflow.log_artifacts(str(results_dir), "evaluation_results")
    
    def _log_config_params(self, config):
        """Log configuration parameters."""
        # Flatten nested config for logging
        flat_params = self._flatten_dict(config)
        
        for key, value in flat_params.items():
            if isinstance(value, (str, int, float, bool)):
                mlflow.log_param(key, value)
    
    def _log_classification_metrics(self, results_dir):
        """Log classification metrics."""
        metrics_dir = Path(results_dir) / "metrics"
        
        for classifier_dir in metrics_dir.glob("*"):
            if classifier_dir.is_dir() and classifier_dir.name != "target_word_evaluation":
                metrics_file = classifier_dir / "metrics.json"
                if metrics_file.exists():
                    with open(metrics_file) as f:
                        metrics = json.load(f)
                    
                    classifier_name = classifier_dir.name
                    test_metrics = metrics.get("test_performance", {})
                    
                    for metric, value in test_metrics.items():
                        mlflow.log_metric(f"{classifier_name}_{metric}", value)
    
    def _log_target_word_metrics(self, results_dir):
        """Log target word evaluation metrics."""
        target_metrics_file = Path(results_dir) / "metrics" / "target_word_evaluation" / "target_word_eval_summary.json"
        
        if target_metrics_file.exists():
            with open(target_metrics_file) as f:
                metrics = json.load(f)
            
            overall_metrics = metrics.get("overall_metrics", {})
            for metric, value in overall_metrics.items():
                if isinstance(value, (int, float)):
                    mlflow.log_metric(f"target_word_{metric}", value)
    
    def _log_trained_models(self, results_dir):
        """Log trained models."""
        models_dir = Path(results_dir) / "models"
        
        for model_file in models_dir.glob("*.pkl"):
            # Load and log model
            import pickle
            with open(model_file, 'rb') as f:
                model = pickle.load(f)
            
            model_name = model_file.stem.replace('_model', '')
            mlflow.sklearn.log_model(model, model_name)
    
    def _flatten_dict(self, d, parent_key='', sep='_'):
        """Flatten nested dictionary."""
        items = []
        for k, v in d.items():
            new_key = f"{parent_key}{sep}{k}" if parent_key else k
            if isinstance(v, dict):
                items.extend(self._flatten_dict(v, new_key, sep=sep).items())
            else:
                items.append((new_key, v))
        return dict(items)

# Usage example
logger = MLflowLogger("mgpt-diabetes-evaluation")
logger.log_evaluation_run(
    job_name="diabetes_eval_20240115",
    config=config_dict,
    results_dir="outputs/diabetes_eval_20240115"
)
```

### Weights & Biases Integration

```python
# utils/wandb_integration.py
import wandb
import json
import pandas as pd
from pathlib import Path

class WandBLogger:
    def __init__(self, project_name="mgpt-eval"):
        self.project_name = project_name
    
    def log_evaluation_run(self, job_name, config, results_dir):
        """Log evaluation run to Weights & Biases."""
        
        run = wandb.init(
            project=self.project_name,
            name=job_name,
            config=config
        )
        
        try:
            # Log metrics and artifacts
            self._log_metrics(results_dir)
            self._log_visualizations(results_dir)
            self._log_data_summary(results_dir)
            
        finally:
            wandb.finish()
    
    def _log_metrics(self, results_dir):
        """Log all metrics to wandb."""
        # Classification metrics
        metrics_dir = Path(results_dir) / "metrics"
        
        for classifier_dir in metrics_dir.glob("*"):
            if classifier_dir.is_dir() and classifier_dir.name != "target_word_evaluation":
                metrics_file = classifier_dir / "metrics.json"
                if metrics_file.exists():
                    with open(metrics_file) as f:
                        metrics = json.load(f)
                    
                    classifier_name = classifier_dir.name
                    wandb.log({f"{classifier_name}/{k}": v for k, v in metrics["test_performance"].items()})
        
        # Target word metrics
        target_file = metrics_dir / "target_word_evaluation" / "target_word_eval_summary.json"
        if target_file.exists():
            with open(target_file) as f:
                target_metrics = json.load(f)
            
            wandb.log({f"target_word/{k}": v for k, v in target_metrics["overall_metrics"].items() if isinstance(v, (int, float))})
    
    def _log_visualizations(self, results_dir):
        """Log plots and visualizations."""
        metrics_dir = Path(results_dir) / "metrics"
        
        # Log confusion matrices and ROC curves
        for classifier_dir in metrics_dir.glob("*"):
            if classifier_dir.is_dir() and classifier_dir.name != "target_word_evaluation":
                
                # Confusion matrix
                cm_file = classifier_dir / "confusion_matrix.png"
                if cm_file.exists():
                    wandb.log({f"{classifier_dir.name}/confusion_matrix": wandb.Image(str(cm_file))})
                
                # ROC curve
                roc_file = classifier_dir / "roc_curve.png"
                if roc_file.exists():
                    wandb.log({f"{classifier_dir.name}/roc_curve": wandb.Image(str(roc_file))})
    
    def _log_data_summary(self, results_dir):
        """Log data summary and predictions."""
        # Log predictions table
        target_predictions = Path(results_dir) / "metrics" / "target_word_evaluation" / "target_word_predictions.csv"
        
        if target_predictions.exists():
            df = pd.read_csv(target_predictions)
            
            # Create wandb table
            table = wandb.Table(dataframe=df.head(100))  # Log first 100 rows
            wandb.log({"target_word_predictions": table})
```

## 🔐 Security & Compliance

### Data Security

#### Encryption at Rest
```bash
# Encrypt sensitive data files
gpg --symmetric --cipher-algo AES256 data/medical_claims.csv

# Decrypt for processing
gpg --decrypt data/medical_claims.csv.gpg > /tmp/claims.csv

# Use temporary file in config
python main.py run-all --config configs/secure_config.yaml

# Clean up
shred -vfz /tmp/claims.csv
```

#### Secure Configuration
```yaml
# secure_config.yaml
model_api:
  base_url: "${MGPT_API_URL}"          # From environment
  headers:
    Authorization: "Bearer ${API_TOKEN}" # Secure token

input:
  dataset_path: "${ENCRYPTED_DATA_PATH}" # Encrypted data

output:
  embeddings_dir: "/secure/outputs/embeddings"
  models_dir: "/secure/outputs/models"

logging:
  level: "INFO"
  file: "/secure/logs/pipeline.log"
  # Disable sensitive data logging
  mask_sensitive_data: true
```

### HIPAA Compliance

#### Audit Logging
```python
# utils/audit_logger.py
import logging
import json
from datetime import datetime
import hashlib

class HIPAAAuditLogger:
    def __init__(self, audit_file="/secure/audit/mgpt_eval_audit.log"):
        self.audit_file = audit_file
        self.logger = logging.getLogger("hipaa_audit")
        
        # Configure audit logger
        handler = logging.FileHandler(audit_file)
        formatter = logging.Formatter('%(asctime)s - %(message)s')
        handler.setFormatter(formatter)
        self.logger.addHandler(handler)
        self.logger.setLevel(logging.INFO)
    
    def log_data_access(self, user_id, dataset_path, action="read"):
        """Log data access for HIPAA compliance."""
        
        # Hash dataset path for privacy
        dataset_hash = hashlib.sha256(dataset_path.encode()).hexdigest()[:16]
        
        audit_entry = {
            "timestamp": datetime.now().isoformat(),
            "event_type": "data_access",
            "user_id": user_id,
            "dataset_hash": dataset_hash,
            "action": action,
            "compliance": "HIPAA"
        }
        
        self.logger.info(json.dumps(audit_entry))
    
    def log_model_inference(self, user_id, sample_count, model_type):
        """Log model inference events."""
        
        audit_entry = {
            "timestamp": datetime.now().isoformat(),
            "event_type": "model_inference",
            "user_id": user_id,
            "sample_count": sample_count,
            "model_type": model_type,
            "compliance": "HIPAA"
        }
        
        self.logger.info(json.dumps(audit_entry))
    
    def log_data_export(self, user_id, export_path, record_count):
        """Log data export events."""
        
        export_hash = hashlib.sha256(export_path.encode()).hexdigest()[:16]
        
        audit_entry = {
            "timestamp": datetime.now().isoformat(),
            "event_type": "data_export",
            "user_id": user_id,
            "export_hash": export_hash,
            "record_count": record_count,
            "compliance": "HIPAA"
        }
        
        self.logger.info(json.dumps(audit_entry))

# Usage in production
audit = HIPAAAuditLogger()
audit.log_data_access("user123", "data/patient_claims.csv", "read")
audit.log_model_inference("user123", 1000, "mgpt_embedding")
```

#### Data Anonymization
```python
# utils/anonymization.py
import pandas as pd
import hashlib
import re

class DataAnonymizer:
    def __init__(self, salt="mgpt_eval_2024"):
        self.salt = salt
    
    def anonymize_dataset(self, input_path, output_path):
        """Anonymize medical claims dataset."""
        
        df = pd.read_csv(input_path)
        
        # Anonymize MCIDs
        df['mcid'] = df['mcid'].apply(self._hash_id)
        
        # Remove any potential PHI patterns
        df['claims'] = df['claims'].apply(self._remove_phi_patterns)
        
        # Save anonymized dataset
        df.to_csv(output_path, index=False)
        
        return output_path
    
    def _hash_id(self, original_id):
        """Create deterministic anonymous ID."""
        hash_input = f"{original_id}{self.salt}"
        return hashlib.sha256(hash_input.encode()).hexdigest()[:12]
    
    def _remove_phi_patterns(self, text):
        """Remove potential PHI patterns from text."""
        
        # Remove social security numbers
        text = re.sub(r'\b\d{3}-\d{2}-\d{4}\b', '[SSN]', text)
        
        # Remove phone numbers
        text = re.sub(r'\b\d{3}-\d{3}-\d{4}\b', '[PHONE]', text)
        
        # Remove dates (be careful not to remove medical codes)
        text = re.sub(r'\b\d{1,2}/\d{1,2}/\d{4}\b', '[DATE]', text)
        
        return text

# Usage
anonymizer = DataAnonymizer()
anonymized_path = anonymizer.anonymize_dataset(
    "data/raw_claims.csv",
    "data/anonymized_claims.csv"
)
```

## 🎯 Best Practices Summary

### Production Deployment Checklist

#### Infrastructure
- [ ] **Containerization**: Docker images for consistent deployment
- [ ] **Orchestration**: Kubernetes/Docker Compose for scaling
- [ ] **Monitoring**: Resource usage and performance metrics
- [ ] **Logging**: Structured logging with appropriate levels
- [ ] **Backup**: Regular backup of results and configurations

#### Security
- [ ] **Data encryption**: At rest and in transit
- [ ] **Access control**: Role-based access to data and systems
- [ ] **Audit logging**: HIPAA-compliant audit trails
- [ ] **Data anonymization**: Remove PHI before processing
- [ ] **Secure secrets**: Environment variables for sensitive data

#### Performance
- [ ] **Resource optimization**: Memory and CPU usage tuning
- [ ] **Batch processing**: Optimal batch sizes for throughput
- [ ] **Parallel processing**: Multi-threading where applicable
- [ ] **Caching**: Cache embeddings and models when possible
- [ ] **Checkpointing**: Resume capability for long-running jobs

#### Quality Assurance
- [ ] **Data validation**: Input data quality checks
- [ ] **Model validation**: Performance threshold monitoring
- [ ] **Result validation**: Output quality verification
- [ ] **Regression testing**: Automated testing pipeline
- [ ] **Documentation**: Comprehensive operational documentation

### Operational Excellence

#### Monitoring & Alerting
```yaml
# monitoring_config.yaml
monitoring:
  metrics:
    - accuracy_threshold: 0.80        # Alert if below 80%
    - processing_time_max: 3600       # Alert if exceeds 1 hour
    - memory_usage_max: 85            # Alert if memory > 85%
    - error_rate_max: 0.05            # Alert if error rate > 5%
  
  notifications:
    email: ["ops@company.com"]
    slack: "#mgpt-alerts"
    pagerduty: "mgpt-service"
```

#### Automated Recovery
```bash
#!/bin/bash
# scripts/auto_recovery.sh

# Monitor evaluation jobs and restart if needed
while true; do
    # Check if evaluation is running
    if ! pgrep -f "main.py run-all" > /dev/null; then
        echo "No evaluation running, checking for pending jobs..."
        
        # Check for pending datasets
        if ls /data/pending/*.csv 1> /dev/null 2>&1; then
            echo "Found pending datasets, starting evaluation..."
            python scripts/process_pending.py
        fi
    fi
    
    sleep 300  # Check every 5 minutes
done
```

### Future Enhancements

1. **Real-time Processing**: Stream processing for live data
2. **Auto-scaling**: Dynamic resource allocation based on load
3. **Multi-model Support**: Compare multiple MGPT model versions
4. **Active Learning**: Identify samples that need manual review
5. **Federated Learning**: Distributed training across institutions
6. **Explainable AI**: Better interpretation of model decisions
7. **AutoML**: Automated hyperparameter optimization
8. **Edge Deployment**: On-premise deployment for sensitive data

## 🔗 Summary

This advanced usage guide covered:

### ✅ Production Deployment
- Docker and Kubernetes deployment strategies
- Environment configuration and secrets management
- Health checks and monitoring setup

### ⚡ Performance Optimization
- Large-scale data processing techniques
- Parallel and distributed processing
- Resource monitoring and adaptive optimization

### 🔄 Automated Workflows
- Scheduled evaluation pipelines
- CI/CD integration with GitHub Actions and Jenkins
- Email and Slack notifications

### 📊 Analytics Integration
- MLflow experiment tracking
- Weights & Biases logging
- Custom metrics dashboards

### 🔐 Security & Compliance
- HIPAA-compliant audit logging
- Data encryption and anonymization
- Secure configuration management

### Quick Start for Production:

```bash
# 1. Create production configuration
cp configs/templates/04_full_pipeline.yaml production_config.yaml

# 2. Set environment variables
export MGPT_API_URL="https://your-mgpt-api.com"
export NOTIFICATION_EMAILS="ops@company.com"

# 3. Deploy with Docker
docker-compose up --build

# 4. Monitor logs
docker-compose logs -f mgpt-eval
```

You now have everything needed to deploy MGPT-Eval in a production environment with enterprise-grade reliability, security, and monitoring!